In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

# Creating the investment analysis crew

## Planner and Replanner

In [3]:
from langchain_openai import ChatOpenAI
planner_llm = ChatOpenAI(model="gpt-5-nano", temperature=0)

In [ ]:
from typing import TypedDict, List, Literal, Union
from pydantic import BaseModel, Field

class Respond(BaseModel):
    response: str
    
class UseAgent(BaseModel):
    agent_to_use: Literal[ "technical_analysis_agent", "fundamental_analysis_agent"] = Field(..., description="Agent to use")
    query_to_send: str = Field(..., description="The query to ask this agent")

class Step(BaseModel):
    step_number: int = Field(..., description="The step number in the plan")
    action: Union[UseAgent, Respond] = Field(
        ..., 
        description= (
            "Action to perform. Use `Respond` if you want to respond to the user "
            "without using a tool. If you need tools, use `UseAgent`. "
        )
    )

class Plan(BaseModel):
    steps: List[Step] = Field(..., description="The list of steps in the plan")

In [21]:
from langchain_core.prompts import ChatPromptTemplate

planner_prompt = ChatPromptTemplate.from_template(
    """For a given objective, come up with a simple step by step plan to achieve the objective.
    
    The plan should involve individual tasks only.
    
    <important>
        Invoke the fundamental_analysis_agent and technical_analysis_agent only once per company of interest.
        If there are two companies of interest, invoke the respective agent/s twice.
        Always use both agents to analyze stock market data when evaluating investments.
    </important>
    
    The user's question is: {objective}
    """
)

llm_planner = planner_prompt | planner_llm.with_structured_output(Plan)
plan = llm_planner.invoke("Based on the past 5 years of stock market data, recommend whether to buy, hold, or sell the stocks of Apple and Tesla.")

In [22]:
plan.steps

[Step(step_number=1, action=UseAgent(agent_to_use='fundamental_analysis_agent', query_to_send='Analyze the fundamentals of Apple (AAPL) using the past 5 years of stock market data, focusing on revenue growth, earnings growth, profit margins, return on equity, balance sheet strength (debt and liquidity), free cash flow, capital allocation (dividends and buybacks), product portfolio, competitive position, management quality, and macro risks. Provide a concise buy/hold/sell rating with rationale and key risks.')),
 Step(step_number=2, action=UseAgent(agent_to_use='technical_analysis_agent', query_to_send='Analyze the technical setup of Apple (AAPL) using daily price data from the past 5 years. Evaluate 50-day and 200-day moving averages, RSI, MACD, volume, support and resistance levels, trend, and chart patterns. Provide a concise buy/hold/sell rating with rationale.')),
 Step(step_number=3, action=UseAgent(agent_to_use='fundamental_analysis_agent', query_to_send='Analyze the fundamentals

### Replanner

In [ ]:
replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
)

llm_replanner = replanner_prompt | planner_llm.with_structured_output(Plan)

In [ ]:
class State(TypedDict):
    query: str
    tickers: List[str]
    plan: Plan
    fundamental_analysis_agent_reply: str
    technical_analysis_agent_reply: str
    search_agent_reply: str
    steps_done: List[Step]